In [ ]:

#Импортируем необходимые библиотеки

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#####################################
from bs4 import BeautifulSoup   
import re 
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


# Читаем тестовые данные и проверяем, все ли загрузилось успешно:

In [ ]:
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3 )

print(train)
print(test)

# Затем напишем функцию, которая уберет из строк лишнее и оставит только текст:

In [ ]:
def check_text(raw_review):
    temp_text = BeautifulSoup(raw_review).get_text()     
    letters = re.sub("[^a-zA-Z]", " ", temp_text) 
    words = letters.lower().split()                             
    stops = set(stopwords.words("english"))                  
    result = [w for w in words if not w in stops]   
    return(" ".join(result))

# Используя функцию выше, очистим текст от лишних тегов и знаков препинания:

In [ ]:
processed_text_train = []
for i in range(0, train["review"].size):
    processed_text_train.append(check_text(train["review"][i]))

# После этого нам нужно получить вектора признаков:

In [ ]:
##########################################
#Получим вектора признаков для train
vectorizer = CountVectorizer(max_features = 5000) 

train_data = vectorizer.fit_transform(processed_text_train)

train_data = train_data.toarray()

# Затем загружаем необходимую технику обучения(в нашем случае это RandomForest):

In [ ]:
r_forest = RandomForestClassifier(n_estimators = 200, random_state = 50) 
r_forest = r_forest.fit(train_data, train["sentiment"])

##########################################
#Аналогично получим вектора признаков для test
processed_text_test = [] 
for i in range(0,test["review"].size):
    processed_text_test.append(check_text(test["review"][i]))
    
test_data = vectorizer.transform(processed_text_test)
test_data = test_data.toarray()
##########################################

result = r_forest.predict(test_data)



# Выводим ответ:

In [ ]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result})
output.to_csv("submission.csv", index=False, quoting=3)